#### generate plate ref frame images

In [ ]:
import numpy as np
import pygplates
import os
from IPython.display import Image
import glob, re
import sys
from velocity_tools import get_velocities
from create_gpml import create_gpml_healpix_mesh
from surface_plotting import create_model_file_list_by_time
from resolve_topologies import resolve_topologies as topology2gmt
from model_config import model_defs 

wencao = False

start_time=-1
end_time=1e7

#if wencao:
#    start_time=249
#    end_time=410

# @_@ LOOK HERE
ModelNum = 'gld428_m21'
#deltas = True
deltas = False

cptfile = 'DT.cpt'
if not deltas:
    os.system('gmt makecpt -Cpolar -T-1200/1200/100 -D > %s' % cptfile)
else:
    os.system('gmt makecpt -Cpolar -T-50/50/5 -D > %s' % cptfile)

root_path = './'
ModelPath = root_path + 'models/'

if not deltas:
    outfile_stem = ModelPath + ModelNum + '/Dynamic_topography/dist/PlateFrame/'
else:
    outfile_stem = ModelPath + ModelNum + '/Dynamic_topography/dist_deltas/PlateFrame/'
    
if not os.path.isdir(outfile_stem):
    os.system('mkdir --parents ' + outfile_stem)
    
file_list = glob.glob(outfile_stem+'*.nc')
model_time_list=[]

points_file = ModelPath + ModelNum + '/Dynamic_topography/PlateFrame/points.xy'
points_cpt_file = ModelPath + ModelNum + '/Dynamic_topography/PlateFrame/points'

for filename in file_list:
    f = os.path.basename(filename)
    f = re.sub(ModelNum, '', f)
    if f.startswith('.'):
        f = f[1:]
        print(f)
    time = float(re.findall(r'\d*\.\d+|\d+',f)[-1])
    model_time_list.append((time,filename))
    model_time_list = sorted(model_time_list, key=lambda model: model[0])

print(model_time_list)

proj = '-JX20c/10c'

for time,grdfile in model_time_list:
    #print os.path.splitext(grdfile)
    #outfile = os.path.splitext(grdfile)[0]
    if "Masked" in os.path.splitext(grdfile)[0]:
        outfile = '%sMasked_%0.2f' % (outfile_stem,time)
    else:
        outfile = '%s%0.2f' % (outfile_stem,time)
    print(outfile)

    #os.system('gmt gmtset COLOR_MODEL RGB BASEMAP_TYPE fancy ANNOT_FONT_SIZE_PRIMARY 12 LABEL_FONT_SIZE 12 HEADER_FONT_SIZE 18 ANNOT_FONT_PRIMARY Helvetica PLOT_DEGREE_FORMAT ddd COLOR_BACKGROUND lightgrey COLOR_FOREGROUND lightgrey COLOR_NAN lightgrey')
    os.system('gmt gmtset COLOR_MODEL RGB MAP_FRAME_TYPE fancy FONT_ANNOT_PRIMARY 10 FONT_LABEL 8 FONT_TITLE 8 FONT_ANNOT_PRIMARY Helvetica FORMAT_GEO_MAP ddd MAP_FRAME_PEN 1.0p')

    os.system('gmt grdimage -C%s %s -E300 -Rd -JX20c/10c -X1c -Y1c -V -P -K > %s.ps' % (cptfile,grdfile,outfile))
    os.system('gmt psclip clip.txt -Rd %s -A -O -K -V >> %s.ps' % (proj,outfile))
    
    if wencao:
        os.system('gmt psxy -Rd %s -Sa0.3 -C%s -O -K %s -V>> %s.ps' % (proj, points_cpt_file, points_file, outfile))
        os.system('gmt psclip clip.txt -Rd %s -A -O -K -V >> %s.ps' % (proj,outfile))

    os.system('gmt pscoast -Dl -R -J -O -K -V -W.25,black >> %s.ps' % outfile)
    os.system('gmt psclip clip.txt -Rd %s -A -O -V >> %s.ps' % (proj,outfile))

    os.system('gmt psconvert %s.ps -A -E240 -Tj -P' % outfile) 
    
    os.system('rm %s.ps' % outfile)

    Image('%s.jpg' % outfile)

print('done')


#### generate mantle ref frame images

In [ ]:
import os, glob
from pathlib import Path

Path("./tmp").mkdir(parents=True, exist_ok=True)

cptfile = 'DT.cpt'
if not deltas:
    os.system('gmt makecpt -Cpolar -T-1200/1200/100 -D > %s' % cptfile)
else:
    os.system('gmt makecpt -Cpolar -T-50/50/5 -D > %s' % cptfile)

root_path = './'
ModelPath = root_path + 'models/'

print(model_defs[ModelNum])

if 'rotations' in model_defs[ModelNum]:
    input_rotation_filename = [root_path + x for x in model_defs[ModelNum]['rotations']]
elif 'rotation_dir' in model_defs[ModelNum]:
    input_rotation_filename = glob.glob(root_path+model_defs[ModelNum]['rotation_dir']+'/*.rot')
    
if 'topology_dir' in model_defs[ModelNum]:
    input_topology_filename = glob.glob(root_path + model_defs[ModelNum]['topology_dir']+'/*.gpml') 
else:
    input_topology_filename = []

print(input_topology_filename)

coastlines_file = root_path + model_defs[ModelNum]['coastlines']
output_coastlines_filename = 'tmp/coastlines.gmt'

output_filename_prefix = 'tmp/'
output_filename_extension = 'gmt'

anchor_plate = 0

if not deltas:
    outfile_stem = ModelPath + ModelNum + '/Dynamic_topography/dist/MantleFrame/'
    mantle_grids = ModelPath + ModelNum + '/Dynamic_topography/MantleFrame/'
else:
    outfile_stem = ModelPath + ModelNum + '/Dynamic_topography/dist_deltas/MantleFrame/'
    mantle_grids = ModelPath + ModelNum + '/Dynamic_topography/dist_deltas/MantleFrame/'
    
if not os.path.isdir(outfile_stem):
    os.system('mkdir --parents ' + outfile_stem)

file_list = glob.glob(mantle_grids+'*.nc')
file_list.extend(glob.glob(mantle_grids+'*.grd'))
model_time_list=[]

for filename in file_list:
    f = os.path.basename(filename)
    f = re.sub(ModelNum, '', f)
    if f.startswith('.'):
        f = f[1:]
        print(f)
    time = float(re.findall(r'\d*\.\d+|\d+',f)[-1])
    if end_time>=time>=start_time:
        model_time_list.append((time,filename))
    model_time_list = sorted(model_time_list, key=lambda model: model[0])   

for time,grdfile in model_time_list:
    os.system('rm -rf tmp/*')
    print(grdfile,time)

    reconstruction_time = time
    outfile = '%s%0.2f' % (outfile_stem,time)

    topology2gmt(input_rotation_filename,
                 input_topology_filename,
                 time,
                 output_filename_prefix,
                 output_filename_extension,
                 anchor_plate)

    pygplates.reconstruct(coastlines_file,input_rotation_filename,output_coastlines_filename,time)
    # NOTE - STILL MISSING OTHER BOUNDARY TYPES

    vel_domain = create_gpml_healpix_mesh(8,filename=None,feature_type='MeshNode')
    rotation_model = pygplates.RotationModel(input_rotation_filename)

    pt_lat,pt_lon,pt_vel1,pt_vel2,plate_ids = get_velocities(
        rotation_model,
        input_topology_filename,
        time,
        velocity_domain_features=vel_domain,
        delta_time=1,
        velocity_type='MagAzim')

    tmp = np.vstack((pt_lon,pt_lat,np.degrees(pt_vel2),np.array(pt_vel1)*0.0025))
    np.savetxt('vtest.xy',tmp.T)
    
    if wencao:
        point_features = []
        
        lines = np.loadtxt(points_file)
        for line in lines:
            point_feature = pygplates.Feature()
            point_feature.set_geometry(pygplates.PointOnSphere(float(line[1]),float(line[0])))
            point_feature.set_name(str(int(line[2])))
            point_features.append(point_feature)
        
        
        assigned_point_features = pygplates.partition_into_plates(
            root_path + model_defs[ModelNum]['static_polygons'],
            rotation_model,
            point_features,
            properties_to_copy = [
                pygplates.PartitionProperty.reconstruction_plate_id,
                pygplates.PartitionProperty.valid_time_period])
        
        assigned_point_feature_collection = pygplates.FeatureCollection(assigned_point_features)
        reconstructed_feature_geometries = []
        pygplates.reconstruct(assigned_point_feature_collection,
            rotation_model,
            reconstructed_feature_geometries,
            time,
            anchor_plate_id=0)
        print(reconstructed_feature_geometries)
        
        with open('points_r.xy',"w+") as f:
            for geom in reconstructed_feature_geometries:
                f.write('{0} {1} {2}\n'.format(
                    geom.get_reconstructed_geometry().to_lat_lon()[1],
                    geom.get_reconstructed_geometry().to_lat_lon()[0],
                    geom.get_feature().get_name()))

    def fix_gmt_file(filename):
        try:
            f = open(filename, "r")
            lines = f.readlines()
        except UnicodeDecodeError:
            f = open(filename, "r", encoding='ISO-8859-1')#temporary fix encoding problem
            lines = f.readlines()
            
        f.close()

        flag = False
        newlines = []
        for idx, line in enumerate(lines):   
            if line.startswith(">"):
                flag = True
            else:
                if flag and not line.startswith("#"):
                    newlines.append('#placeholder\n')
                flag = False
        
            newlines.append(line)
                
        f = open(filename, "w", encoding='utf-8')
        f.writelines(newlines)
        f.close()
    
    proj = '-JX20c/10c'

    os.system('gmt grdimage %s -C%s %s -E300 -Rd -X1c -Y1c -V -P -K > %s.ps' % (proj,cptfile,grdfile,outfile))
    os.system('gmt psclip clip.txt -Rd %s -A -O -V -K >> %s.ps' % (proj,outfile))
    
    fix_gmt_file('tmp/coastlines.gmt')
    #fix_gmt_file('tmp/coastlines/coastlines_polyline.gmt')
    #fix_gmt_file('tmp/coastlines/coastlines_polyline.gmt')
    os.system('gmt psxy -Rd %s -W0.25p,grey10 -O -K tmp/coastlines.gmt -V >> %s.ps' % (proj,outfile))
    #os.system('gmt psclip clip.txt -Rd %s -A -O -V -K >> %s.ps' % (proj,outfile))

    #os.system('gmt psxy -Rd %s -W0.25p,grey10 -O -K -m tmp/coastlines/coastlines_polyline.gmt -V >> %s.ps' % (proj,outfile))
    #os.system('gmt psclip clip.txt -Rd %s -A -O -V -K >> %s.ps' % (proj,outfile))
    
    #os.system('gmt psxy -Rd %s -W0.25p,grey10 -O -K -m tmp/coastlines/coastlines_polygon.gmt -V >> %s.ps' % (proj,outfile))
    #os.system('gmt psclip clip.txt -Rd %s -A -O -V -K >> %s.ps' % (proj,outfile))
    
    #os.system('gmt psxy -Rd %s -W0.25p,grey10 -O -K -m tmp/coastlines/coastlines_point.gmt -V >> %s.ps' % (proj,outfile))
    #os.system('gmt psclip clip.txt -Rd %s -O -V -K >> %s.ps' % (proj,outfile))

    fix_gmt_file('tmp/ridge_transform_boundaries_%0.2fMa.gmt' % reconstruction_time)
    os.system('gmt psxy -Rd %s -W0.6p,red -O -K tmp/ridge_transform_boundaries_%0.2fMa.gmt -V >> %s.ps' \
              % (proj,reconstruction_time,outfile))
    #os.system('gmt psclip clip.txt -Rd %s -A -O -V -K >> %s.ps' % (proj,outfile))
    
    fix_gmt_file('tmp/subduction_boundaries_sL_%0.2fMa.gmt' % reconstruction_time)
    os.system('gmt psxy -Rd %s -W0.6p,blue -Gblue -Sf10p/2.5p+l+t -K -O tmp/subduction_boundaries_sL_%0.2fMa.gmt -V >> %s.ps' \
              % (proj,reconstruction_time,outfile))
    #os.system('gmt psclip clip.txt -Rd %s -A -O -V -K >> %s.ps' % (proj,outfile))

    fix_gmt_file('tmp/subduction_boundaries_sR_%0.2fMa.gmt' % reconstruction_time)
    os.system('gmt psxy -Rd %s -W0.6p,blue -Gblue -Sf10p/2.5p+r+t -K -O tmp/subduction_boundaries_sR_%0.2fMa.gmt -V >> %s.ps' \
              % (proj,reconstruction_time,outfile))
    #os.system('gmt psclip clip.txt -Rd %s -A -O -V -K >> %s.ps' % (proj,outfile))
    
    if not wencao:
        os.system('gmt psxy vtest.xy -Rd %s -W0.5p,gray30 -SV0.075+e+a45 -V -Ggray30 -O -K >> %s.ps' % (proj,outfile))
    else:
        os.system('gmt psxy -Rd %s -Sa0.3 -C%s -O %s -V -K >> %s.ps' % (proj, points_cpt_file, 'points_r.xy', outfile))

    #os.system('gmt psclip clip.txt -Rd %s -A -O -V >> %s.ps' % (proj,outfile))
    os.system(f'gmt psclip -C -O >> {outfile}.ps')

    os.system('gmt psconvert %s.ps -A -E240 -Tj -P' % outfile) 
    
    os.system('rm %s.ps' % outfile)
    
    Image('%s.jpg' % outfile)
   
print('done')

In [ ]:
try:
    f = open('./tmp/subduction_boundaries_sL_560.00Ma.gmt', "r")
    lines = f.readlines()
except UnicodeDecodeError:
    f = open('./tmp/subduction_boundaries_sL_560.00Ma.gmt', "r", encoding='ISO-8859-1')
    lines = f.readlines()